In [996]:
import numpy as np
import pandas as pd
import pybaseball
import seaborn as sns
import matplotlib.pyplot as plot
from deepdiff import DeepDiff
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register
from pybaseball import pitching_stats
from pybaseball import batting_stats
from time import sleep

cache.enable()
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 100)

# These are the people

In [997]:
people = pd.DataFrame(people())

sleep(1)  
# sleep calls to prevent the pybaseball scraper 
# from throwing errors when importing tons of stuff

people = people.convert_dtypes() # cleanup; thank you NumPy

people = people[people.weight.notna()]

print(people.shape)
print("---")
people.sample(3)

(19729, 24)
---


,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
14764,porceri01,1988,12,27,USA,NJ,Morristown,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Rick,Porcello,Frederick Alfred,205,77,R,R,2009-04-09,2020-09-26,porcr001,porceri01
17511,speiech01,1950,6,28,USA,CA,Alameda,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Chris,Speier,Chris Edward,175,73,R,R,1971-04-07,1989-10-01,speic001,speiech01
12411,metzgbu01,1952,5,23,USA,IN,Lafayette,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Butch,Metzger,Clarence Edward,185,73,R,R,1974-09-08,1978-06-28,metzb101,metzgbu01


We need to add more ID info about them so that we don't drown in merge errors.

In [998]:
rosetta = pd.DataFrame(chadwick_register()) 

sleep(1)

rosetta = rosetta.convert_dtypes()

rosetta = rosetta.dropna(how='any')

print(rosetta.shape)
print('---')
rosetta.sample(3)

(20506, 8)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
8716,Haddix,Harvey,115289,haddh101,haddiha01,1005184,1952,1965
12012,Krug,Chris,117333,krugc101,krugch01,1007178,1965,1969
12718,Linebrink,Scott,276530,lines001,linebsc01,564,2000,2011


## Merge people into rosetta

In [999]:
everyone = pd.merge(rosetta, people, left_on='key_bbref', right_on='playerID', how='left')

print(everyone.shape)
print('---')
everyone.sample(3)

(20506, 32)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
5979,Fitzgerald,John,114192,fitzj101,fitzgjo02,1004109,1890,1890,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3921,Courtright,John,112740,courj001,courtjo01,1002679,1995,1995,courtjo01,1970,5,30,USA,OH,Marion,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,John,Courtright,John Charles,185,74,L,L,1995-05-06,1995-05-06,courj001,courtjo01
14854,Pounds,Bill,120709,pounb101,poundbi01,1010478,1903,1903,poundbi01,1878,3,11,USA,NJ,Paterson,1936,7,7,USA,NJ,Paterson,Bill,Pounds,Jeared Wells,178,70,R,L,1903-05-02,1903-06-30,pounb101,poundbi01


## Init Fielding Data

In [1000]:
# fielding stats by year 
fielding = pd.DataFrame(fielding()) # this is fine

fielding = fielding.convert_dtypes()

print(fielding.shape)
print('---')
fielding.sample(3)

(147080, 18)
---


,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
19696,fienelo01,1908,1,CHA,AL,P,1,1,27,0,5,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
3249,salisha01,1882,1,PT1,AA,P,38,<NA>,1005,10,96,11,1,<NA>,<NA>,<NA>,<NA>,<NA>
83302,flynndo01,1984,1,MON,NL,2B,88,78,2013,148,223,8,47,<NA>,<NA>,<NA>,<NA>,<NA>


In [1001]:
fielding = fielding.rename(columns={"playerID": "key_bbref"})

In [1002]:
print(list(everyone.columns))

['name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_bbref', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last', 'playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID']


In [1003]:
everyone = everyone[
    [
        "name_last",
        "name_first",
        "key_mlbam",
        "key_retro",
        "key_bbref",
        "key_fangraphs",
        "birthYear",
        "deathYear",
        "weight",
        "height",
        "bats",
        "throws",
    ]
]

In [1004]:
print(everyone.shape)
print('---')
everyone.sample(10)

(20506, 12)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,birthYear,deathYear,weight,height,bats,throws
6256,Frederick,Johnny,114395,fredj101,fredejo01,1004311,1902,1977,165,71,L,L
13807,Ogden,Curly,119951,ogdec101,ogdencu01,1009742,1901,1964,180,73,R,R
13789,Ockey,Walter,119915,ockew101,ockeywa01,1009706,1920,1971,175,72,R,R
20502,Zuvella,Paul,124789,zuvep001,zuvelpa01,1014465,1958,<NA>,173,72,R,R
11168,Lynch,Ed,118029,lynce001,lynched01,1007862,1956,<NA>,230,78,R,R
14415,Perdomo,Luis,466412,perdl001,perdolu01,5380,1984,<NA>,180,72,R,R
18076,Suzuki,Mac,123010,suzum001,suzukma01,1485,1975,<NA>,195,75,R,R
47,Acevedo,Jose,406895,acevj002,acevejo01,307,1977,<NA>,185,72,R,R
2564,Buschhorn,Don,111790,buscd101,buschdo01,1001746,1946,<NA>,170,72,R,R
15861,Rohe,George,121397,roheg101,rohege01,1011145,1874,1957,165,69,R,R


In [1005]:
df = everyone
df = fielding.merge(df, on='key_bbref', how='outer', validate='many_to_one')

print(df.shape)
print('---')
df.head(10)

(147865, 29)
---


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws
0,abercda01,1871,1,TRO,<NA>,SS,1,1,24,1,3,2,0,<NA>,<NA>,<NA>,<NA>,<NA>,Abercrombie,Frank,110018,aberd101,1000017,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,addybo01,1871,1,RC1,<NA>,2B,22,22,606,67,72,42,5,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
2,addybo01,1871,1,RC1,<NA>,SS,3,3,96,8,14,7,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
3,addybo01,1873,1,PH2,<NA>,2B,10,<NA>,249,24,23,8,2,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
4,addybo01,1873,2,BS1,<NA>,OF,31,<NA>,843,30,3,14,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
5,addybo01,1874,1,HR1,<NA>,2B,45,45,1197,125,139,48,7,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
6,addybo01,1874,1,HR1,<NA>,3B,5,4,171,10,10,15,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
7,addybo01,1874,1,HR1,<NA>,SS,1,1,12,0,1,2,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
8,addybo01,1875,1,PH2,<NA>,2B,2,<NA>,48,3,7,3,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
9,addybo01,1875,1,PH2,<NA>,OF,68,<NA>,1800,91,14,33,2,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L


In [1006]:
print(list(df.columns))
print(df.shape)
df.sample(10)

['key_bbref', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR', 'name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_fangraphs', 'birthYear', 'deathYear', 'weight', 'height', 'bats', 'throws']
(147865, 29)


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws
142208,lugose01,2017,1,NYN,NL,P,19,18,304,8,7,0,1,<NA>,<NA>,<NA>,<NA>,<NA>,Lugo,Seth,607625,lugos001,12447,1989,<NA>,233,76,R,R
112,ansonca01,1895,1,CHN,NL,1B,122,<NA>,3150,1176,60,19,82,<NA>,<NA>,<NA>,<NA>,<NA>,Anson,Cap,110284,ansoc101,1000272,1852,1922,227,72,R,R
60302,spangal01,1959,1,ML1,NL,OF,4,2,71,6,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Spangler,Al,122560,spana101,1012302,1933,<NA>,175,72,L,L
1651,nelsoca01,1872,2,BR1,<NA>,3B,4,3,75,5,7,4,0,<NA>,<NA>,<NA>,<NA>,<NA>,Nelson,Candy,119705,nelsc101,1009498,1849,1910,145,66,L,R
101660,vandejo02,1993,1,MON,NL,1B,42,31,655,237,13,3,17,<NA>,<NA>,<NA>,<NA>,<NA>,Vander Wal,John,123639,vandj001,853,1966,<NA>,180,73,L,L
147797,surhob.01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Surhoff,B. J.,122989,surhb001,117,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
69168,otisam01,1967,1,NYN,NL,OF,16,16,419,22,2,0,1,<NA>,<NA>,<NA>,<NA>,<NA>,Otis,Amos,120111,otisa001,1009893,1947,<NA>,165,71,R,R
7229,doughch01,1884,1,ALT,UA,SS,1,<NA>,6,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Dougherty,Charlie,113493,dougc101,1003428,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
76857,poqueto01,1978,1,KCA,AL,OF,63,54,1269,144,5,7,0,<NA>,<NA>,<NA>,<NA>,<NA>,Poquette,Tom,120674,poqut101,1010443,1951,<NA>,175,70,L,R
109243,holtch01,2001,1,DET,AL,P,30,22,454,9,17,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,Holt,Chris,116108,holtc001,1005983,1971,<NA>,205,76,R,R


In [1007]:
print(df.shape)
print('---')
df.head(10)

(147865, 29)
---


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws
0,abercda01,1871,1,TRO,<NA>,SS,1,1,24,1,3,2,0,<NA>,<NA>,<NA>,<NA>,<NA>,Abercrombie,Frank,110018,aberd101,1000017,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,addybo01,1871,1,RC1,<NA>,2B,22,22,606,67,72,42,5,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
2,addybo01,1871,1,RC1,<NA>,SS,3,3,96,8,14,7,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
3,addybo01,1873,1,PH2,<NA>,2B,10,<NA>,249,24,23,8,2,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
4,addybo01,1873,2,BS1,<NA>,OF,31,<NA>,843,30,3,14,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
5,addybo01,1874,1,HR1,<NA>,2B,45,45,1197,125,139,48,7,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
6,addybo01,1874,1,HR1,<NA>,3B,5,4,171,10,10,15,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
7,addybo01,1874,1,HR1,<NA>,SS,1,1,12,0,1,2,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
8,addybo01,1875,1,PH2,<NA>,2B,2,<NA>,48,3,7,3,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L
9,addybo01,1875,1,PH2,<NA>,OF,68,<NA>,1800,91,14,33,2,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,L,L


## Init Batting data

In [1008]:
# batting stats by year 
batting = pd.DataFrame(batting()) # this is fine

batting = batting.convert_dtypes()
batting.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

print(batting.shape)
print('---')
batting.sample(3)

(110495, 22)
---


,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
81002,evereca01,2001,1,BOS,AL,102,409,61,105,24,4,14,58,9,2,27,104,3,13,0,0,3
64262,youmafl01,1986,1,MON,NL,33,75,4,12,0,0,1,7,0,0,5,29,0,0,2,0,0
80551,wardbr01,2000,1,PHI,NL,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [1009]:
batting = batting.rename(columns={"playerID": "key_bbref"})

In [1010]:
batting.head(3)

,key_bbref,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,<NA>,1,4,0,0,0,0,0,0,0,0,0,0,<NA>,<NA>,<NA>,<NA>,0
1,addybo01,1871,1,RC1,<NA>,25,118,30,32,6,0,0,13,8,1,4,0,<NA>,<NA>,<NA>,<NA>,0
2,allisar01,1871,1,CL1,<NA>,29,137,28,40,4,5,0,19,3,1,2,5,<NA>,<NA>,<NA>,<NA>,1


### Merge Batting Data

In [1011]:
a = np.intersect1d(df.columns, batting.columns)
print(list(a))

['CS', 'G', 'SB', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID']


In [1012]:
df = df.merge(batting, on=['CS', 'G', 'SB', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator=True)

print(df.shape)
print('---')
df.sample(30)

(256732, 44)
---


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws,AB,R,H,2B,3B,HR,RBI,BB,SO,IBB,HBP,SH,SF,GIDP,_merge
82546,cruzto02,1983,1,SEA,AL,SS,63,62,1625,97,224,12,42,<NA>,<NA>,<NA>,<NA>,<NA>,Cruz,Todd,112907,cruzt001,1002844,1955,2008,175,72,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
163992,dubucje01,1918,1,BOS,AL,<NA>,5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6,0,1,0,0,0,0,1,2,<NA>,0,0,<NA>,<NA>,right_only
109080,glanvdo01,1999,1,PHI,NL,OF,148,146,3803,385,13,8,3,<NA>,<NA>,<NA>,<NA>,<NA>,Glanville,Doug,114844,gland001,956,1970,<NA>,170,74,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
155787,colliji01,1903,1,BOS,AL,<NA>,130,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,23,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,540,88,160,33,17,5,72,24,28,<NA>,2,13,<NA>,<NA>,right_only
74868,cosgrmi01,1974,1,HOU,NL,P,45,0,270,1,13,1,2,<NA>,<NA>,<NA>,<NA>,<NA>,Cosgrove,Mike,112711,cosgm101,1002651,1951,<NA>,170,73,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
235374,bazaryo01,2007,1,DET,AL,<NA>,11,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,0,0,0,0,right_only
175819,mustaal01,1940,1,BOS,AL,<NA>,6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6,0,2,1,0,0,3,0,2,<NA>,0,0,<NA>,0,right_only
148668,kesslhe01,1877,1,CN1,NL,<NA>,6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20,0,2,0,0,0,0,2,1,<NA>,<NA>,<NA>,<NA>,<NA>,right_only
217987,maddugr01,1993,1,ATL,NL,<NA>,36,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,91,5,15,1,0,0,4,1,32,0,0,10,0,0,right_only
100743,hentgpa01,1995,1,TOR,AL,P,30,30,602,12,18,2,2,<NA>,<NA>,<NA>,<NA>,<NA>,Hentgen,Pat,115797,hentp001,145,1968,<NA>,210,74,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only


In [1013]:
print(list(df.columns))
df.sample(10)

['key_bbref', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR', 'name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_fangraphs', 'birthYear', 'deathYear', 'weight', 'height', 'bats', 'throws', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP', '_merge']


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws,AB,R,H,2B,3B,HR,RBI,BB,SO,IBB,HBP,SH,SF,GIDP,_merge
41094,fletcel01,1939,2,PIT,NL,1B,101,<NA>,<NA>,1010,56,8,97,<NA>,<NA>,<NA>,<NA>,<NA>,Fletcher,Elbie,114235,flete101,1004150,1916,1994,180,72,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
215654,hornsa01,1991,1,BAL,AL,<NA>,121,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,317,45,74,16,0,23,61,41,99,4,3,0,1,10,right_only
122048,gonzaad01,2007,1,SDN,NL,1B,161,161,4388,1470,140,10,134,<NA>,<NA>,<NA>,<NA>,<NA>,Gonzalez,Adrian,408236,gonza003,1908,1982,<NA>,215,74,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
77312,almonbi01,1985,1,PIT,NL,OF,32,23,507,27,2,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Almon,Bill,110181,almob001,1000174,1952,<NA>,180,75,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
14108,malarjo01,1903,1,BSN,NL,P,32,<NA>,759,16,75,11,5,<NA>,<NA>,<NA>,<NA>,<NA>,Malarkey,John,118185,malaj102,1008011,1872,1949,155,71,<NA>,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
7581,harkijo01,1885,1,BR3,AA,P,34,<NA>,879,36,57,10,0,<NA>,<NA>,<NA>,<NA>,<NA>,Harkins,John,115477,harkj101,1005364,1859,1940,205,73,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
244923,wachami01,2013,1,SLN,NL,<NA>,15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,21,0,3,0,0,0,3,0,10,0,0,1,0,0,right_only
103008,revento01,1992,1,OAK,AL,P,2,0,6,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Revenig,Todd,121091,revet001,1010854,1969,<NA>,185,73,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
71377,hebneri01,1983,1,PIT,NL,3B,40,28,842,16,43,2,1,<NA>,<NA>,<NA>,<NA>,<NA>,Hebner,Richie,115686,hebnr001,1005571,1947,<NA>,195,73,L,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
224254,reedje02,1998,1,COL,NL,<NA>,113,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,259,43,75,17,1,9,39,37,57,4,1,3,3,6,right_only


## Init Pitching data

In [1014]:
# pitching stats by year 
pitching = pd.DataFrame(pitching()) # this is fine

sleep(1)

pitching = pitching.convert_dtypes()

print(pitching.shape)
print('---')
pitching.sample(10)

(49430, 30)
---


,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
7475,gearidi01,1924,2,BSN,NL,0,1,1,1,0,0,0,0,3,5,0,2,0,1.0,<NA>,<NA>,0,0,0,5,0,5,<NA>,<NA>,<NA>
22886,rileyge01,1979,1,CHN,NL,0,1,4,1,0,0,0,39,16,8,1,6,5,0.32,5.54,1,0,2,0,59,0,9,1,0,1
28088,learyti01,1991,1,NYA,AL,4,10,28,18,1,0,0,362,150,87,20,57,83,0.312,6.49,1,10,4,0,551,4,89,7,2,13
16333,jayjo01,1961,1,CIN,NL,21,10,34,34,14,4,0,742,217,97,25,92,157,0.23,3.53,1,9,5,0,1032,0,102,<NA>,<NA>,<NA>
37426,rosarfr01,2006,1,TOR,AL,1,2,17,1,0,0,0,69,24,17,4,16,21,0.264,6.65,2,3,1,0,108,4,17,0,0,1
4934,brownbu01,1913,1,BSN,NL,0,0,2,0,0,0,0,40,19,7,0,3,3,0.39,4.73,<NA>,0,2,0,55,1,10,<NA>,<NA>,<NA>
23304,sarmima01,1980,1,SEA,AL,0,1,9,0,0,0,1,44,14,6,2,6,15,0.255,3.68,1,0,0,0,63,6,7,0,2,0
9643,leebi02,1934,1,CHN,NL,13,14,35,29,16,4,1,643,218,81,9,74,104,0.26,3.4,<NA>,9,2,1,910,4,91,<NA>,<NA>,<NA>
40764,kontoge01,2011,1,NYA,AL,0,0,7,0,0,0,0,18,4,2,1,3,6,0.19,3.0,0,0,0,0,24,4,2,0,0,0
13393,epperal01,1950,1,BRO,NL,0,0,5,0,0,0,0,27,14,5,1,5,3,0.37,5.0,<NA>,0,0,0,44,2,8,<NA>,<NA>,<NA>


In [1015]:
pitching = pitching.rename(columns={"playerID": "key_bbref"})

In [1016]:
a = np.intersect1d(df.columns, pitching.columns)
print(list(a))

['BB', 'G', 'GIDP', 'GS', 'H', 'HBP', 'HR', 'IBB', 'R', 'SF', 'SH', 'SO', 'WP', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID']


In [1017]:
df = df.merge(pitching, on=['BB', 'G', 'GIDP', 'GS', 'H', 'HBP', 'HR', 'IBB', 'R', 'SF', 'SH', 'SO', 'WP', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator="second_merge")

print(df.shape)
print('---')


(306162, 57)
---


## FIP, wRC+

In [1018]:
# bwar_bat stats by year 
bwar_bat = pd.DataFrame(bwar_bat()) # this is fine

bwar_bat = bwar_bat.convert_dtypes()

print(bwar_bat.shape)
print('---')
bwar_bat.sample(10)

(119731, 17)
---


,name_common,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,pitcher,G,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR
15157,Bobby Byrne,111841,byrnebo01,1912,PIT,1,NL,N,130,599,3600,5.4,10.4,-0.6,1.75,0.47,2.22
40052,Mike Goodfellow,114949,goodfmi01,1888,CLE,1,AA,N,68,283,<NA>,-11.3,-10.3,-4.5,0.63,-1.05,-0.42
116147,Poindexter Williams,<NA>,willipo02,1930,NEG,2,NNL,N,12,46,<NA>,4.1,4.1,0.4,0.16,0.36,0.52
21773,Brian Cooper,150001,coopebr01,1999,ANA,1,AL,Y,0,0,<NA>,0.0,0.0,0.0,<NA>,<NA>,<NA>
66338,Matt Mantei,118256,mantema01,2005,BOS,1,AL,Y,5,0,750000,0.0,0.0,0.0,0.0,0.0,0.0
31911,Felix Evans,<NA>,evansfe01,1941,MRS,0,NAL,N,2,4,<NA>,-0.9,-0.9,-0.1,0.01,-0.1,-0.09
81371,Charlie Pabor,120150,paborch01,1872,CLE,1,<NA>,N,21,92,<NA>,-8.9,-8.9,-1.3,0.27,-0.58,-0.31
89446,Jerry Reuss,121090,reussje01,1975,PIT,1,NL,Y,32,86,<NA>,3.1,3.1,0.2,0.0,0.33,0.33
73634,Clarence Mitchell,119189,mitchcl01,1924,PHI,1,NL,Y,69,107,4700,-0.2,-0.2,0.0,0.07,-0.04,0.03
1370,Jack Allen,110141,allenja01,1879,SYR,1,NL,N,11,49,<NA>,-5.6,-1.6,-3.9,0.18,-0.53,-0.35


In [1019]:
bwar_bat = bwar_bat.rename(columns={
    "year_ID": "yearID",
    "player_ID": "key_bbref",
    "team_ID": "teamID",
    "lg_ID": "lgID",
})

In [1020]:
a = np.intersect1d(df.columns, bwar_bat.columns)
print(list(a))

['G', 'key_bbref', 'lgID', 'teamID', 'yearID']


In [1021]:
df = df.merge(bwar_bat, on=['G', 'key_bbref', 'lgID', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator="third_merge")

In [1022]:
print(df.shape)
df.sample(30)

(377003, 70)


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws,AB,R,H,2B,3B,HR,RBI,BB,SO,IBB,HBP,SH,SF,GIDP,_merge,W,L,CG,SHO,SV,IPouts,ER,BAOpp,ERA,BK,BFP,GF,second_merge,name_common,mlb_ID,stint_ID,pitcher,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,third_merge
339668,kessido01,1967,<NA>,CHC,NL,<NA>,145,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Don Kessinger,117024,1,N,628,<NA>,-13.3,-19.2,14.4,2.22,-1.4,0.82,right_only
315041,cabreda01,2008,<NA>,BAL,AL,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Daniel Cabrera,425555,1,Y,5,2875000,-0.7,-0.7,0.0,0.0,-0.08,-0.08,right_only
324477,eppsha01,1940,<NA>,STL,NL,<NA>,11,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Hal Epps,113870,1,N,15,<NA>,-2.7,-1.7,-1.0,0.06,-0.29,-0.23,right_only
251253,pfefffr01,1883,1,CHN,NL,<NA>,96,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,371,41,87,22,7,1,45,8,50,<NA>,<NA>,<NA>,<NA>,<NA>,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
9378,foutzda01,1888,1,BR3,AA,P,23,<NA>,528,18,45,5,4,<NA>,<NA>,<NA>,<NA>,<NA>,Foutz,Dave,114346,foutd101,1004266,1856,1897,161,74,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
329458,gonzama02,2019,<NA>,SEA,AL,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Marco Gonzales,594835,1,Y,6,900000,1.6,1.6,0.0,0.0,0.14,0.14,right_only
331696,hamilda02,1999,<NA>,NYM,NL,<NA>,55,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Darryl Hamilton,115378,2,N,189,<NA>,7.4,9.3,-0.8,0.57,0.64,1.21,right_only
361387,russebr01,1929,<NA>,SLS,NNL,<NA>,91,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Branch Russell,<NA>,0,N,406,<NA>,13.2,13.2,-3.6,1.34,1.23,2.57,right_only
231430,garcile02,2013,2,CHA,AL,OF,6,4,114,9,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Garcia,Leury,544725,garcl004,5913,1991,<NA>,190,68,B,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
286354,gerenbo01,1990,1,NYA,AL,<NA>,110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,277,21,59,7,0,8,31,13,73,1,5,6,2,7,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only


In [1023]:
# bwar_pit stats by year 
bwar_pitch = pd.DataFrame(bwar_pitch()) # this is fine

bwar_pitch = bwar_pitch.convert_dtypes()

print(bwar_pitch.shape)
print('---')
bwar_pitch.sample(10)

(53669, 19)
---


,name_common,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G,GS,RA,xRA,BIP,BIP_perc,salary,ERA_plus,WAR_rep,WAA,WAA_adj,WAR
15741,Bob Friend,114442,frienbo01,1961,PIT,1,NL,41,35,119,120.642,832,0.1845,<NA>,102.736634,2.5075,0.2993,-0.1025,2.7
38687,Pat Ragan,120885,raganpa01,1914,BRO,1,NL,38,25,104,90.188,706,0.1542,<NA>,95.542029,1.7799,-1.0944,-0.1124,0.57
12339,Dave Dowling,113511,dowlida01,1966,CHC,1,NL,1,1,2,3.93,34,0.0072,<NA>,192.45,0.0855,0.3038,-0.0054,0.38
24217,Jim Kaat,116803,kaatji01,1983,STL,1,NL,24,0,19,16.08,128,0.0262,<NA>,94.526667,0.3331,-0.4848,-0.0078,-0.16
37655,Juan Pizarro,120612,pizarju01,1967,PIT,1,NL,50,9,55,45.576,306,0.0645,<NA>,84.904255,1.0739,-1.34,-0.6856,-0.95
51481,Howard Wilson,124402,wilsohi01,1902,PHA,1,AL,13,10,44,53.587,366,0.0848,<NA>,150.576923,0.9633,1.1271,-0.1104,1.98
9427,Ryan Cook,475857,cookry01,2013,OAK,1,AL,71,0,22,32.371,198,0.0448,505000,153.415789,0.6796,0.8236,0.0943,1.6
33558,Carlos Muniz,444370,munizca01,2008,NYM,1,NL,18,0,14,11.889,75,0.0167,<NA>,79.171429,0.2046,-0.315,0.0133,-0.1
24025,Slim Jones,<NA>,jonessl01,1934,PS,0,NN2,30,22,44,111.002,572,0.293,<NA>,333.146429,2.0162,6.891,-0.4594,8.45
3613,Lou Bierbauer,110981,bierblo01,1886,PHA,1,AA,2,0,9,6.753,44,0.0096,<NA>,85.0,0.0643,-0.1386,-0.0135,-0.09


In [1024]:
bwar_pitch = bwar_pitch.rename(columns={
    "year_ID": "yearID",
    "player_ID": "key_bbref",
    "team_ID": "teamID",
    "lg_ID": "lgID",
})

In [1025]:
a = np.intersect1d(df.columns, bwar_pitch.columns)
print(list(a))

['G', 'GS', 'WAA', 'WAR', 'WAR_rep', 'key_bbref', 'lgID', 'mlb_ID', 'name_common', 'salary', 'stint_ID', 'teamID', 'yearID']


In [1026]:
df = df.merge(bwar_pitch, on=['G', 'GS', 'WAA', 'WAR', 'WAR_rep', 'key_bbref', 'lgID', 'mlb_ID', 'name_common', 'salary', 'stint_ID', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator="fourth_merge")

In [1027]:
print(df.shape)
df.sample(30)

(430632, 77)


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws,AB,R,H,2B,3B,HR,RBI,BB,SO,IBB,HBP,SH,SF,GIDP,_merge,W,L,CG,SHO,SV,IPouts,ER,BAOpp,ERA,BK,BFP,GF,second_merge,name_common,mlb_ID,stint_ID,pitcher,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,third_merge,RA,xRA,BIP,BIP_perc,ERA_plus,WAA_adj,fourth_merge
67451,nichobi01,1953,1,PHI,NL,OF,12,<NA>,<NA>,13,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Nicholson,Bill,119774,nichb101,1009554,1914,1996,205,72,L,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
196698,uribeju01,2011,1,LAN,NL,3B,59,53,1388,37,96,3,3,<NA>,<NA>,<NA>,<NA>,<NA>,Uribe,Juan,346874,uribj002,454,1979,<NA>,245,72,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
417631,rodriwa01,2012,<NA>,PIT,NL,<NA>,13,12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Wandy Rodriguez,434643,2,<NA>,<NA>,10000000,<NA>,<NA>,<NA>,0.6422,0.2379,0.84,NaN,33,34.829,242,0.0554,101.448387,-0.0379,right_only
175143,alfoned01,1996,1,NYN,NL,SS,15,8,255,7,22,0,5,<NA>,<NA>,<NA>,<NA>,<NA>,Alfonzo,Edgardo,110135,alfoe001,859,1973,<NA>,210,71,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
14118,farredu01,1891,1,BS2,AA,3B,66,<NA>,1647,87,160,22,8,<NA>,<NA>,<NA>,<NA>,<NA>,Farrell,Duke,114019,farrd102,1003942,1866,1925,208,73,B,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
189419,kimby01,2005,1,COL,NL,<NA>,40,22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,11,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,82,156,<NA>,<NA>,17,<NA>,71,115,8,14,8,7,11,NaN,5,12,0,0,0,444,80,0.275,4.86,1,667,3,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
248873,whitemi03,2021,1,LAN,NL,<NA>,21,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,15,1,0,0,0,0,0,1,11,0,0,0,0,0,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
60344,bridgto01,1946,1,DET,AL,<NA>,9,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,16,24,<NA>,<NA>,5,<NA>,8,17,<NA>,1,<NA>,<NA>,<NA>,NaN,1,1,0,0,1,64,14,0.27,5.91,0,95,6,right_only,Tommy Bridges,111458,1,Y,5,8000,0.3,0.3,0.0,0.0,0.03,0.03,both,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
181,battijo01,1874,1,PH1,<NA>,SS,5,5,129,10,11,9,0,<NA>,<NA>,<NA>,<NA>,<NA>,Battin,Joe,110691,battj102,1000672,1853,1937,169,70,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
28262,camniho01,1911,1,PIT,NL,P,40,<NA>,<NA>,4,59,5,1,<NA>,<NA>,<NA>,<

### Clean-up

In [1028]:
print(list(df.columns))

['key_bbref', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR', 'name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_fangraphs', 'birthYear', 'deathYear', 'weight', 'height', 'bats', 'throws', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP', '_merge', 'W', 'L', 'CG', 'SHO', 'SV', 'IPouts', 'ER', 'BAOpp', 'ERA', 'BK', 'BFP', 'GF', 'second_merge', 'name_common', 'mlb_ID', 'stint_ID', 'pitcher', 'PA', 'salary', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAR_rep', 'WAA', 'WAR', 'third_merge', 'RA', 'xRA', 'BIP', 'BIP_perc', 'ERA_plus', 'WAA_adj', 'fourth_merge']


In [1029]:
print(df.shape)
df.sample(30)

(430632, 77)


,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws,AB,R,H,2B,3B,HR,RBI,BB,SO,IBB,HBP,SH,SF,GIDP,_merge,W,L,CG,SHO,SV,IPouts,ER,BAOpp,ERA,BK,BFP,GF,second_merge,name_common,mlb_ID,stint_ID,pitcher,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,third_merge,RA,xRA,BIP,BIP_perc,ERA_plus,WAA_adj,fourth_merge
88898,sullifr01,1958,1,BOS,AL,<NA>,32,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,67,6,11,1,0,0,3,5,18,0,0,8,0,2,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,Frank Sullivan,122945,1,Y,80,<NA>,-1.5,-1.5,0.0,0.0,-0.18,-0.18,both,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
35914,meloapa01,1911,2,SLA,AL,OF,54,<NA>,<NA>,69,6,8,1,<NA>,<NA>,<NA>,<NA>,<NA>,Meloan,Paul,118936,melop101,1008751,1888,1950,175,70,L,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
93081,mccormi03,1968,1,SFN,NL,P,38,28,595,9,23,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,McCormick,Mike,118598,mccom103,1008420,1938,2020,195,74,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
346400,maynabu01,1943,<NA>,NYG,NL,<NA>,121,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Buster Maynard,118488,1,N,424,<NA>,-27.7,-19.7,-10.0,1.43,-3.28,-1.85,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
411136,nenro01,1993,<NA>,FLA,NL,<NA>,15,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Robb Nen,119718,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.2879,-1.032,-0.53,NaN,28,17.305,112,0.0239,61.176923,0.2114,right_only
256015,lordbr01,1910,1,CLE,AL,<NA>,58,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,210,23,46,8,7,0,17,12,27,<NA>,2,6,<NA>,<NA>,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,Bris Lord,117924,1,N,234,<NA>,-8.9,-7.9,-4.0,0.97,-1.15,-0.18,both,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
189181,hergema01,2003,2,SFN,NL,<NA>,27,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,11,28,<NA>,<NA>,1,<NA>,9,28,0,1,1,1,5,NaN,1,0,0,0,0,105,9,0.219,2.31,1,140,3,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
355138,pedroeu01,1920,<NA>,CSW,NNL,<NA>,49,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Eustaquio Pedroso,<NA>,0,N,203,<NA>,-9.0,-7.6,-3.3,0.71,-0.93,-0.22,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
106250,mikkepe01,1972,1,LAN,NL,<NA>,33,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7,0,0,0,0,0,0,1,6,0,0,0,0,0,right_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
98747,mcauldi01,1964,1,DET,AL,SS,160,159,4239,262,467,32,84,<NA>,<

In [1030]:
everyone = df[
    [
        "key_bbref",
        "name_last",
        "name_first",
        "weight",
        "height",
        "yearID",
        "POS",
        "WAR",
        "WAA",
        "pitcher",
        "ERA_plus",
        "birthYear",
        "deathYear",
        "teamID",
        "lgID",
        "runs_above_avg",
        "runs_above_avg_def",
        "WAA_adj",
        "G",
        "GS",
        "InnOuts",
        "PO",
        "A",
        "E",
        "DP",
        "PB",
        "WP",
        "SB",
        "CS",
        "ZR",
        "bats",
        "throws",
        "AB",
        "R",
        "H",
        "2B",
        "3B",
        "HR",
        "RBI",
        "BB",
        "SO",
        "IBB",
        "HBP",
        "SH",
        "SF",
        "GIDP",
        "W",
        "L",
        "CG",
        "SHO",
        "SV",
        "IPouts",
        "ER",
        "BAOpp",
        "ERA",
        "BK",
        "BFP",
        "GF",
        "name_common",
        "mlb_ID",
        "stint_ID",
        "PA",
        "salary",
        "runs_above_avg_off",
        "WAR_rep",
        "RA",
        "xRA",
        "BIP",
        "BIP_perc",
        "_merge",
        "second_merge",
        "third_merge",
        "fourth_merge",
        "key_mlbam",
        "key_retro",
        "key_fangraphs",
    ]
]

In [1031]:
everyone.loc[everyone['key_bbref'] == 'colonba01']

,key_bbref,name_last,name_first,weight,height,yearID,POS,WAR,WAA,pitcher,ERA_plus,birthYear,deathYear,teamID,lgID,runs_above_avg,runs_above_avg_def,WAA_adj,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,bats,throws,AB,R,H,2B,3B,HR,RBI,BB,SO,IBB,HBP,SH,SF,GIDP,W,L,CG,SHO,SV,IPouts,ER,BAOpp,ERA,BK,BFP,GF,name_common,mlb_ID,stint_ID,PA,salary,runs_above_avg_off,WAR_rep,RA,xRA,BIP,BIP_perc,_merge,second_merge,third_merge,fourth_merge,key_mlbam,key_retro,key_fangraphs
182390,colonba01,Colon,Bartolo,285,71,1997,P,<NA>,<NA>,<NA>,<NA>,1973,<NA>,CLE,AL,<NA>,<NA>,<NA>,19,17,282,6,17,5,3,<NA>,<NA>,<NA>,<NA>,<NA>,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,left_only,left_only,left_only,112526,colob001,375
182391,colonba01,<NA>,<NA>,<NA>,<NA>,1997,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CLE,AL,<NA>,<NA>,<NA>,19,17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,66,107,<NA>,<NA>,12,<NA>,45,66,1,3,4,1,11,4,7,1,0,0,282,59,0.286,5.65,0,427,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,right_only,left_only,left_only,<NA>,<NA>,<NA>
182392,colonba01,<NA>,<NA>,<NA>,<NA>,1997,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CLE,AL,<NA>,<NA>,<NA>,19,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,1,0,0,0,0,0,0,0,1,0,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,right_only,left_only,left_only,left_only,<NA>,<NA>,<NA>
182393,colonba01,Colon,Bartolo,285,71,1998,P,<NA>,<NA>,<NA>,<NA>,1973,<NA>,CLE,AL,<NA>,<NA>,<NA>,31,31,612,13,34,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,R,R,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,left_only,left_only,left_only,112526,colob001,375
182394,colonba01,<NA>,<NA>,<NA>,<NA>,1998,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CLE,AL,<NA>,<NA>,<NA>,31,31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,91,205,<NA>,<NA>,15,<NA>,79,158,5,3,10,2,17,14,9,6,2,0,612,84,0.26,3.71,0,883,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,right_only,left_only,left_only,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386165,colonba01,<NA>,<NA>,<NA>,<NA>,2015,<NA>,1.29,-0.2838,<NA>,91.365556,<NA>,<NA>,NYM,NL,<NA>,<NA>,-0.1506,33,31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Bartolo Colon,112526,1,<NA>,11000000,<NA>,1.7231,94,88.484,651,0.1521,NaN,NaN,NaN,right_only,<NA>,<NA>,<NA>
386166,colonba01,<NA>,<NA>,<NA>,<NA>,2016,<NA>,2.95,1.3974,<NA>,117.063014,<NA>,<NA>,NYM,NL,<NA>,<NA>,-0.1486,34,33,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Bartolo Colon,112526,1,<NA>,7250000,<NA>,1.6982,81,92.269,628,0.1496,NaN,NaN,NaN,right_only,<NA>,<NA>,<NA>
386167,colonba01,<NA>,<NA>,<NA>,<NA>,2017,<NA>,-2.14,-2.6546,<NA>,53.603509,<NA>,<NA>,ATL,NL,<NA>,<NA>,-0.0449,13,13,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Bartolo Colon,112526,1,<NA>,12500000,<NA>,0.563,66,32.574,236,0.0537,NaN,NaN,NaN,right_only,<NA>,<NA>,<NA>
386168,colonba01,<NA>,<NA>,<NA>,<NA>,2017,<NA>,0.27,-0.4575,<NA>,85.773913,<NA>,<NA>,MIN,AL,<NA>,<NA>,-0.0541,15,15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

In [1032]:
print(list(everyone.columns))

['key_bbref', 'name_last', 'name_first', 'weight', 'height', 'yearID', 'POS', 'WAR', 'WAA', 'pitcher', 'ERA_plus', 'birthYear', 'deathYear', 'teamID', 'lgID', 'runs_above_avg', 'runs_above_avg_def', 'WAA_adj', 'G', 'GS', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR', 'bats', 'throws', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP', 'W', 'L', 'CG', 'SHO', 'SV', 'IPouts', 'ER', 'BAOpp', 'ERA', 'BK', 'BFP', 'GF', 'name_common', 'mlb_ID', 'stint_ID', 'PA', 'salary', 'runs_above_avg_off', 'WAR_rep', 'RA', 'xRA', 'BIP', 'BIP_perc', '_merge', 'second_merge', 'third_merge', 'fourth_merge', 'key_mlbam', 'key_retro', 'key_fangraphs']


In [1033]:
df = everyone[
    [
        "key_bbref",
        "weight",
        "height",
        "name_last",
        "name_first",
        "POS",
        "yearID",
        "birthYear",
        "deathYear",
        "teamID",
        "lgID",
    ]
]

# Here it is:

In [1034]:
df = df.sort_values('weight', ascending=False)

In [1035]:
df = df.drop_duplicates(subset=['key_bbref'])

In [1036]:
df.head(30)

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID
208622,youngwa01,320,77,Young,Walter,1B,2005,1980,2015,BAL,AL
234215,diazju03,315,76,Diaz,Jumbo,P,2015,1984,<NA>,CIN,NL
54897,brownju01,295,76,Brown,Jumbo,P,1933,1907,1966,NYA,AL
181856,youngdm01,295,74,Young,Dmitri,1B,1997,1973,<NA>,SLN,NL
198981,rauchjo01,290,83,Rauch,Jon,P,2002,1978,<NA>,CHA,AL
236232,vargake01,290,77,Vargas,Kennys,1B,2014,1990,<NA>,MIN,AL
205773,broxtjo01,285,76,Broxton,Jonathan,P,2014,1984,<NA>,CIN,NL
194742,dunnad01,285,78,Dunn,Adam,1B,2014,1979,<NA>,CHA,AL
195887,paronch01,285,77,Paronto,Chad,P,2009,1975,<NA>,HOU,NL
182432,colonba01,285,71,Colon,Bartolo,P,2011,1973,<NA>,NYA,AL


In [1037]:
# Constants
KG_TO_LB = 0.453592
M_TO_IN = 0.0254

In [1038]:
# BMI Calculations
df['KG'] = df['weight'] * KG_TO_LB
df['meters'] = df['height'] * M_TO_IN
df['BMI'] = df['KG'] / df['meters'] ** 2
df['ratio'] = df['meters'] * df['BMI']


After some noodling, this is the best combo of BMI:height that I could find such that we have enough players at each position to fill out a roster. 2B and 1B ended up being a bottleneck, which isn't surprising. Second-basemen are typically the smallest player on the team, and First-basemen are almost universally the tallest fielder on the roster.

In [1039]:
tryouts = df[ # Let's separate the meat from the chaff
    (df.BMI >= df.BMI.quantile(0.99))
    & (df.height <= df.height.quantile(0.23))]

In [1040]:
tryouts.POS.value_counts()

P     17
C     15
1B     8
OF     7
2B     3
SS     1
3B     1
Name: POS, dtype: Int64

In [1041]:
# Probably easiest if we make ourselves a dataframe for every position, since we'll need to fill up a 26-man roster, which is parsed into several discrete roles that each require a particular number of players. 

# These role counts aren't codified, but teams have nearly always carried the same distribution of player-roles.

# The standard breakdown is:
# 13 Pitchers (5x SP, 7x RP)
# 5 OF
# 2 C
# 4 IF
# 2 Utility

# For Utility players, we'll likely take two extra middle infielders, because middle infielders can usually play any field position without looking like they have no idea what they're doing. E.g., you can put a shortstop at first base, and he'll be a pretty mediocre-to-bad first baseman — but if you put a first baseman at shortstop, he's likely to spend the entire game crying.

tryouts_P = tryouts[tryouts['POS'] == "P"]
tryouts_C = tryouts[tryouts['POS'] == "C"]
tryouts_1B = tryouts[tryouts['POS'] == "1B"]
tryouts_2B = tryouts[tryouts['POS'] == "2B"]
tryouts_3B = tryouts[tryouts['POS'] == "3B"]
tryouts_SS = tryouts[tryouts['POS'] == "SS"]
tryouts_OF = tryouts[tryouts['POS'] == "OF"]

In [1042]:
huskies_P = (
    tryouts_P.sort_values("BMI", ascending=False)
    .nlargest(13, "BMI")
)  # 13 heaviest P
huskies_P

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
182432,colonba01,285,71,Colon,Bartolo,P,2011,1973,<NA>,NYA,AL,129.27372,1.8034,39.748992,71.683331
243208,moronre01,265,70,Moronta,Reyes,P,2017,1993,<NA>,SFN,NL,120.20188,1.778,38.02312,67.605107
216625,mijarjo01,265,71,Mijares,Jose,P,2011,1984,<NA>,MIN,AL,120.20188,1.8034,36.959589,66.652922
3994,healeto01,155,55,Healey,Tom,P,1878,1853,1891,PRO,NL,70.30676,1.397,36.025023,50.326958
229336,machije01,257,71,Machi,Jean,P,2014,1982,<NA>,SFN,NL,116.573144,1.8034,35.843828,64.640759
199068,rodnefe01,240,71,Rodney,Fernando,P,2002,1977,<NA>,DET,AL,108.86208,1.8034,33.472835,60.364911
236741,castran01,240,71,Castro,Angel,P,2015,1982,<NA>,OAK,AL,108.86208,1.8034,33.472835,60.364911
248116,colined01,240,71,Colina,Edwar,P,2020,1997,<NA>,MIN,AL,108.86208,1.8034,33.472835,60.364911
245959,valdefr01,239,71,Valdez,Framber,P,2021,1993,<NA>,HOU,AL,108.408488,1.8034,33.333365,60.11339
224531,carigan01,235,71,Carignan,Andrew,P,2012,1986,<NA>,OAK,AL,106.59412,1.8034,32.775484,59.107308


In [1043]:
huskies_P = (
    tryouts_P.sort_values("BMI", ascending=False)
    .nlargest(13, "BMI")
)  # 13 heaviest P
huskies_P

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
182432,colonba01,285,71,Colon,Bartolo,P,2011,1973,<NA>,NYA,AL,129.27372,1.8034,39.748992,71.683331
243208,moronre01,265,70,Moronta,Reyes,P,2017,1993,<NA>,SFN,NL,120.20188,1.778,38.02312,67.605107
216625,mijarjo01,265,71,Mijares,Jose,P,2011,1984,<NA>,MIN,AL,120.20188,1.8034,36.959589,66.652922
3994,healeto01,155,55,Healey,Tom,P,1878,1853,1891,PRO,NL,70.30676,1.397,36.025023,50.326958
229336,machije01,257,71,Machi,Jean,P,2014,1982,<NA>,SFN,NL,116.573144,1.8034,35.843828,64.640759
199068,rodnefe01,240,71,Rodney,Fernando,P,2002,1977,<NA>,DET,AL,108.86208,1.8034,33.472835,60.364911
236741,castran01,240,71,Castro,Angel,P,2015,1982,<NA>,OAK,AL,108.86208,1.8034,33.472835,60.364911
248116,colined01,240,71,Colina,Edwar,P,2020,1997,<NA>,MIN,AL,108.86208,1.8034,33.472835,60.364911
245959,valdefr01,239,71,Valdez,Framber,P,2021,1993,<NA>,HOU,AL,108.408488,1.8034,33.333365,60.11339
224531,carigan01,235,71,Carignan,Andrew,P,2012,1986,<NA>,OAK,AL,106.59412,1.8034,32.775484,59.107308


In [1052]:
huskies_C = (
    tryouts_C.sort_values("BMI", ascending=False)
    .nlargest(2, "BMI")
)  # 13 heaviest P
huskies_C

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
248415,kirkal01,245,68,Kirk,Alejandro,C,2021,1998,<NA>,TOR,AL,111.13004,1.7272,37.25171,64.341153
207632,penabr01,240,69,Pena,Brayan,C,2016,1982,<NA>,SLN,NL,108.86208,1.7526,35.441412,62.114618


In [1045]:
huskies_1B = (
    tryouts_1B.sort_values("BMI", ascending=False)
    .nlargest(1, "BMI")
)  # 13 heaviest P
huskies_1B

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
217568,sandopa01,268,70,Sandoval,Pablo,1B,2010,1986,<NA>,SFN,NL,121.562656,1.778,38.45357,68.370448


In [1046]:
huskies_2B = (
    tryouts_2B.sort_values("BMI", ascending=False)
    .nlargest(2, "BMI")
)  # 13 heaviest P
huskies_2B

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
241635,whitety01,238,71,White,Tyler,2B,2017,1990,<NA>,HOU,AL,107.954896,1.8034,33.193895,59.86187
232566,phegljo01,225,70,Phegley,Josh,2B,2013,1988,<NA>,CHA,AL,102.0582,1.778,32.283781,57.400562


In [1047]:
huskies_SS = (
    tryouts_SS.sort_values("BMI", ascending=False)
    .nlargest(2, "BMI")
)  # 2 heaviest SS
huskies_SS

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
184377,tejadmi01,220,69,Tejada,Miguel,SS,1997,1974,<NA>,OAK,AL,99.79024,1.7526,32.487961,56.9384


In [1048]:
huskies_3B = (
    tryouts_3B.sort_values("BMI", ascending=False)
    .nlargest(1, "BMI")
)  # 1 heaviest 3B
huskies_3B

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
208969,callaal01,210,68,Callaspo,Alberto,3B,2010,1983,<NA>,KCA,AL,95.25432,1.7272,31.930037,55.14956


In [1051]:
huskies_OF = (
    tryouts_OF.sort_values("BMI", ascending=False)
    .nlargest(5, "BMI")
)  # 13 heaviest P
huskies_OF

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,KG,meters,BMI,ratio
247295,naylojo01,250,71,Naylor,Josh,OF,2020,1997,<NA>,SDN,NL,113.398,1.8034,34.867537,62.880115
223992,vicieda01,240,71,Viciedo,Dayán,OF,2012,1989,<NA>,CHA,AL,108.86208,1.8034,33.472835,60.364911
247419,ramirha02,232,70,Ramirez,Harold,OF,2021,1994,<NA>,CLE,AL,105.233344,1.778,33.288165,59.186358
244258,astudwi01,225,69,Astudillo,Willians,OF,2021,1991,<NA>,MIN,AL,102.0582,1.7526,33.226324,58.232455
51268,fothebo01,230,70,Fothergill,Bob,OF,1923,1897,1938,DET,AL,104.32616,1.778,33.001198,58.67613
